In [4]:
from eva_cttv_pipeline.clinvar_xml_utils import *

import os
import sys
from urllib import request
import xml.etree.ElementTree as ElementTree

import hgvs
import numpy as np
import pandas as pd

In [6]:
sys.path.append('../')
from filter_clinvar_xml import filter_xml

In [7]:
pd.set_option('display.max_colwidth', None)

In [8]:
PROJECT_ROOT = '/home/april/projects/opentargets'
clinvar_path = os.path.join(PROJECT_ROOT, 'ClinVarFullRelease_00-latest.xml.gz')

In [7]:
# Only run this cell if you want to refresh clinvar data

# clinvar_url = 'https://ftp.ncbi.nlm.nih.gov/pub/clinvar/xml/ClinVarFullRelease_00-latest.xml.gz'
# request.urlretrieve(clinvar_url, clinvar_path)
# request.urlcleanup()

## ClinVar documentation

Submission guidelines can be found [here](https://www.ncbi.nlm.nih.gov/clinvar/docs/submit/#required), in particular note the requirement:

> a valid description of the variant, one of:
>    * an HGVS expression
>    * chromosome coordinates and change
>    * cytogenetic description

Also found the [xsd](https://ftp.ncbi.nlm.nih.gov/pub/clinvar/clinvar_submission.xsd) for ClinVar XML submissions, which includes all possible measure types:

```xml
<xs:simpleType name="Measuretype">
    <xs:restriction base="xs:string">
        <xs:enumeration value="Gene"/>
        <xs:enumeration value="Variation"/>
        <xs:enumeration value="Insertion"/>
        <xs:enumeration value="Mobile element insertion"/>
        <xs:enumeration value="Novel sequence insertion"/>
        <xs:enumeration value="Microsatellite"/>
        <xs:enumeration value="Deletion"/>
        <xs:enumeration value="single nucleotide variant"/>
        <xs:enumeration value="Multiple nucleotide variation"/>
        <xs:enumeration value="Indel"/>
        <xs:enumeration value="Duplication"/>
        <xs:enumeration value="Tandem duplication"/>
        <xs:enumeration value="copy number loss"/>
        <xs:enumeration value="copy number gain"/>
        <xs:enumeration value="protein only"/>
        <xs:enumeration value="Inversion"/>
        <xs:enumeration value="Translocation"/>
        <xs:enumeration value="Interchromosomal breakpoint"/>
        <xs:enumeration value="Intrachromosomal breakpoint"/>
        <xs:enumeration value="Complex"/>
    </xs:restriction>
</xs:simpleType>
```
Compare measure types we've actually found in the data, [here](https://github.com/EBIvariation/eva-opentargets/tree/master/data-exploration/clinvar-variant-types).

## Filter

Filter full dataset to get just a manageable (hopefully representative) sample of records with measures representing complex events

In [19]:
complex_xml = os.path.join(PROJECT_ROOT, 'complex-events.xml.gz')

# get just "complex events"
# Q: what's complex? -- complex == no full coordinates
def complex_measures(x):
    if x.measure:
        return (
            # smattering of all non SNV variants
            (x.measure.variant_type.lower() not in {'single nucleotide variant'} and np.random.random() < 0.01)
            # be sure to get the rare ones
            or (x.measure.variant_type.lower() in {'tandem duplication', 'fusion', 'complex', 'translocation', 'inversion'})
        )
    return False

filter_xml(
    input_xml=clinvar_path,
    output_xml=complex_xml,
    filter_fct=complex_measures,
)

Records written: 3040


In [6]:
dataset = ClinVarDataset(complex_xml)

## Dataframe

Load measures into a dataframe where columns are the properties.

In [7]:
def get_measures(dataset):
    for r in dataset:
        if r.measure:
            yield r.measure

In [ ]:
for m in get_measures(dataset):
    break
    
dir(m)

In [8]:
# just all the properties
props = [
 'all_names',
 'clinvar_record',
#  'explicit_insertion_length',
#  'has_complete_coordinates',
 'hgnc_ids',
 'hgvs',
 'is_repeat_expansion_variant',
#  'measure_xml',
 'microsatellite_category',
 'nsv_id',
 'preferred_gene_symbols',
#  'preferred_name',
 'preferred_or_other_name',
#  'pubmed_refs',
 'rs_id',
#  'sequence_location_helper',
#  'toplevel_refseq_hgvs',
 'variant_type',
 'chr',
 'vcf_alt',
#  'vcf_full_coords',
 'vcf_pos',
 'vcf_ref'
]

In [9]:
# replaces empty list with None
measures = [[getattr(v, p) if getattr(v, p) != [] else None for p in props] for v in get_measures(dataset)]

In [10]:
df = pd.DataFrame(measures, columns=props)

In [11]:
df.count()

all_names                      2999
clinvar_record                 3040
hgnc_ids                       2610
hgvs                           2392
is_repeat_expansion_variant    3040
microsatellite_category         208
nsv_id                          272
preferred_gene_symbols         2613
preferred_or_other_name        2999
rs_id                           996
variant_type                   3040
chr                            2033
vcf_alt                        1830
vcf_pos                        1830
vcf_ref                        1830
dtype: int64

In [12]:
set(df['variant_type'])

{'Complex',
 'Deletion',
 'Duplication',
 'Indel',
 'Insertion',
 'Inversion',
 'Microsatellite',
 'Tandem duplication',
 'Translocation',
 'Variation',
 'copy number gain',
 'copy number loss',
 'fusion',
 'protein only'}

In [23]:
df[df.variant_type == 'Translocation']

,all_names,clinvar_record,hgnc_ids,hgvs,is_repeat_expansion_variant,microsatellite_category,nsv_id,preferred_gene_symbols,preferred_or_other_name,rs_id,variant_type,chr,vcf_alt,vcf_pos,vcf_ref
1,[46;XX;t(1;5)(p32.1;q31.3)],ClinVarRecord object with accession RCV000258575,None,None,False,None,None,None,46;XX;t(1;5)(p32.1;q31.3),None,Translocation,None,None,None,None
2,[46;XY;t(1;15)(p22.3;q15)],ClinVarRecord object with accession RCV000258594,None,None,False,None,None,None,46;XY;t(1;15)(p22.3;q15),None,Translocation,None,None,None,None
3,[46;XX;t(2;11)(p13.1;p15.5)dn],ClinVarRecord object with accession RCV000258610,None,None,False,None,None,None,46;XX;t(2;11)(p13.1;p15.5)dn,None,Translocation,None,None,None,None
4,[46;XY;t(5;8)(q15;q22.1)],ClinVarRecord object with accession RCV000258613,None,None,False,None,None,None,46;XY;t(5;8)(q15;q22.1),None,Translocation,None,None,None,None
5,[46;XY;t(5;15)(q13.1;q13)],ClinVarRecord object with accession RCV000258628,None,None,False,None,None,None,46;XY;t(5;15)(q13.1;q13),None,Translocation,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2732,[t(6;9)(q23.3;p22.3)],ClinVarRecord object with accession RCV000585780,"[HGNC:7545, HGNC:7785]",None,False,None,None,"[MYB, NFIB]",t(6;9)(q23.3;p22.3),None,Translocation,None,None,None,None
2766,[t(X;8)(p22.13;q12.1)],ClinVarRecord object with accession RCV000767228,"[HGNC:28509, HGNC:20626]",None,False,None,None,"[BEND2, CHD7]",t(X;8)(p22.13;q12.1),None,Translocation,None,None,None,None
2767,[t(8;9)(q24.22;q34.13)],ClinVarRecord object with accession RCV000767229,"[HGNC:12362, HGNC:26572]",None,False,None,None,"[TSC1, TMEM71]",t(8;9)(q24.22;q34.13),None,Translocation,None,None,None,None
2772,[t(5;22)(q31.2;q12.1)],ClinVarRecord object with accession RCV000785874,None,None,False,None,None,None,t(5;22)(q31.2;q12.1),None,Translocation,None,None,None,None


## XML

Probably not all elements from the raw xml are captured in the object, can we get find anything there?

In [14]:
def get_measure_xml_for_rcv(dataset, rcv):
    for r in dataset:
        if r.accession == rcv:
            return r.measure.measure_xml


# pretty print xml
def pprint(x):
    print(ElementTree.tostring(x, encoding='unicode'))


def print_measure_xml_for_rcv(dataset, rcv):
    x = get_measure_xml_for_rcv(dataset, rcv)
    pprint(x)

In [37]:
xml = get_measure_xml_for_rcv(dataset, 'RCV001372309')
pprint(xml)

<Measure Type="Duplication" ID="1039262">
        <Name>
          <ElementValue Type="Preferred">NC_000023.10:g.(?_31514885)_(32430050_?)dup</ElementValue>
        </Name>
        <AttributeSet>
          <Attribute Accession="NC_000023" Version="10" Change="g.(?_31514885)_(32430050_?)dup" Type="HGVS, genomic, top level, previous" integerValue="37">NC_000023.10:g.(?_31514885)_(32430050_?)dup</Attribute>
        </AttributeSet>
        <CytogeneticLocation>Xp21.1</CytogeneticLocation>
        <SequenceLocation Assembly="GRCh37" AssemblyAccessionVersion="GCF_000001405.25" AssemblyStatus="previous" Chr="X" Accession="NC_000023.10" innerStart="31514885" innerStop="32430050" display_start="31514885" display_stop="32430050" variantLength="915166" />
        <MeasureRelationship Type="within single gene">
          <Name>
            <ElementValue Type="Preferred">dystrophin</ElementValue>
          </Name>
          <Symbol>
            <ElementValue Type="Preferred">DMD</ElementValue>
    

In [34]:
xml2 = get_measure_xml_for_rcv(dataset, 'RCV001255994')
pprint(xml2)

<Measure Type="Inversion" ID="966181">
        <Name>
          <ElementValue Type="Preferred">NM_001098.3(ACO2):c.433-2_433-1inv</ElementValue>
        </Name>
        <CanonicalSPDI>NC_000022.11:41511873:AG:CT</CanonicalSPDI>
        <AttributeSet>
          <Attribute Accession="NM_001098" Version="3" Change="c.433-2_433-1inv" Type="HGVS, coding, RefSeq" MANESelect="true">NM_001098.3:c.433-2_433-1inv</Attribute>
        </AttributeSet>
        <AttributeSet>
          <Attribute Change="g.47750_47751delinsCT" Accession="NG_032143" Version="1" Type="HGVS, genomic">NG_032143.1:g.47750_47751delinsCT</Attribute>
        </AttributeSet>
        <AttributeSet>
          <Attribute Accession="NG_032143" Version="1" Change="g.47750_47751delinsCT" Type="HGVS, genomic, RefSeqGene">NG_032143.1:g.47750_47751delinsCT</Attribute>
        </AttributeSet>
        <AttributeSet>
          <Attribute Accession="NG_032143" Version="1" Change="g.47750_47751inv" Type="HGVS, genomic, RefSeqGene">NG_03214

## Summary for ticket

### ClinVar documentation

Submission guidelines can be found [here](https://www.ncbi.nlm.nih.gov/clinvar/docs/submit/#required), in particular note the requirement:
> * a valid description of the variant, one of:
>    * an HGVS expression
>    * chromosome coordinates and change
>    * cytogenetic description

### Notes from 4 August meeting

Tentatively define "complex events" as variants without full `chr_pos_ref_alt` coordinates, as these are variants that aren't handled by the core VEP-based functional consequence pipeline.  Some of these *are* handled by the specialised repeat expansion pipeline, so we can focus on the rest.

To really deal with complex events, besides identifiers we also need to get *functional consequences*, otherwise they get dropped by the pipeline.

Next steps:
1. Collect more representative examples of complex events in ClinVar
1. Look at distribution of records with no functional consequences
    * How many have HGVS-esque identifiers, sequence location with start/stop position, cytogenetic location?
    * Of those with no full coordinates, how many can we parse? How many can the official HGVS parser parse?
    * How are unparseable HGVS identifiers failing?  Is there some systematic way people are using HGVS-esque identifiers that aren't _technically_ HGVS-compliant but we can still parse reliably?
1. What does VEP require to get consequences? Biologically we should only need start/stop coordinates.
1. Can we report variant/phenotype associations without functional consequences, similar to ontology mappings?  Is this still valuable to OT?
    * Can we report functional consequences with some uncertainty?  For example some HGVS identifiers encode uncertain location, e.g. [here](https://www.ncbi.nlm.nih.gov/clinvar/variation/1062574/).

(1) and (2) I've started doing and will post in a subsequent comment before the next meeting.